In [22]:
#Побудуйте власну рекомендаційну систему (приклад є в документації). Оцініть її. Отримайте рекомендацію для певного користувача.
!pip install scikit-Surprise
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import train_test_split
import numpy as np

In [23]:
class MyOwnAlgorithm(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.user_means = {}
        for uid, ratings in trainset.ur.items():
            self.user_means[uid] = np.mean([rating for (_, rating) in ratings])
        return self

    def estimate(self, uid, iid):
        if uid in self.user_means:
            return self.user_means[uid]
        else:
            return self.trainset.global_mean

In [24]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

model = MyOwnAlgorithm()
model.fit(trainset)

In [25]:
from surprise import accuracy
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.0387
MAE:  0.8325


In [26]:
user_id = str(244)

all_items = set(trainset.all_items())
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Топ-10 фільмів, рекомендованих для користувача:
Фільм 474 з прогнозованим рейтингом 3.66
Фільм 620 з прогнозованим рейтингом 3.66
Фільм 405 з прогнозованим рейтингом 3.66
Фільм 513 з прогнозованим рейтингом 3.66
Фільм 770 з прогнозованим рейтингом 3.66
Фільм 26 з прогнозованим рейтингом 3.66
Фільм 207 з прогнозованим рейтингом 3.66
Фільм 588 з прогнозованим рейтингом 3.66
Фільм 1067 з прогнозованим рейтингом 3.66
Фільм 664 з прогнозованим рейтингом 3.66


In [ ]:
#Висновок: У ході виконання лабораторної роботи було створено просту рекомендаційну систему для передбачення рейтингу фільмів користувачем.
#Спочатку було використано вбудований алгоритм SVD з бібліотеки Surprise, що дозволило нам швидко побудувати модель, навчену на датасеті MovieLens, і оцінити її точність за допомогою метрик RMSE та MAE.
#Після цього, для виконання завдання щодо створення власного алгоритму, було реалізовано простий підхід на основі середніх оцінок користувача.
#Створений алгоритм було перевірено на тому ж наборі даних, і результати моделі були оцінені аналогічним чином.
#Власний алгоритм мав гірші показники точності порівняно з SVD, що підкреслює переваги більш складних моделей для систем рекомендацій.
#Крім цього, для конкретного користувача було отримано список фільмів, рекомендованих на основі передбачених рейтингів, що є ключовим результатом системи рекомендацій.
#Таким чином, у ході лабораторної роботи було успішно реалізовано власний алгоритм для побудови рекомендаційної системи та продемонстровано його роботу на реальних даних.

#Посилання на гітхаб: https://github.com/nastiadzen/Machine-learning